In [58]:
from langchain_huggingface import ChatHuggingFace, HuggingFaceEndpoint
from langchain_core.tools import tool
from dotenv import load_dotenv

In [59]:
from langchain_classic.agents import create_react_agent, AgentExecutor

In [60]:
from langchain_community.tools import DuckDuckGoSearchRun

search = DuckDuckGoSearchRun()

In [61]:
# Step 1: making a tool
@tool
def searchDuckDuck(query: str) -> str:
    """this is a duckduckgo query searching dunction"""
    results = search.invoke(query)
    return results


In [62]:
# Step 2: Pull the ReAct prompt from LangChain Hub
from langchain_classic import hub

#importing prompt template of react from langchain hub
prompt = hub.pull("hwchase17/react")

In [63]:
prompt

PromptTemplate(input_variables=['agent_scratchpad', 'input', 'tool_names', 'tools'], input_types={}, partial_variables={}, metadata={'lc_hub_owner': 'hwchase17', 'lc_hub_repo': 'react', 'lc_hub_commit_hash': 'd15fe3c426f1c4b3f37c9198853e4a86e20c425ca7f4752ec0c9b0e97ca7ea4d'}, template='Answer the following questions as best you can. You have access to the following tools:\n\n{tools}\n\nUse the following format:\n\nQuestion: the input question you must answer\nThought: you should always think about what to do\nAction: the action to take, should be one of [{tool_names}]\nAction Input: the input to the action\nObservation: the result of the action\n... (this Thought/Action/Action Input/Observation can repeat N times)\nThought: I now know the final answer\nFinal Answer: the final answer to the original input question\n\nBegin!\n\nQuestion: {input}\nThought:{agent_scratchpad}')

In [64]:
llm = HuggingFaceEndpoint(
    repo_id="deepseek-ai/DeepSeek-V3.2",
    task ="text-generation",
)
model = ChatHuggingFace(llm = llm)

In [65]:
# Step 3: Create the ReAct agent manually with the pulled prompt
agent = create_react_agent(
    llm= model,
    tools= [searchDuckDuck],
    prompt= prompt,
)

In [66]:
# Step 4: Wrap it with AgentExecutor
agent_executor = AgentExecutor(
    agent = agent,
    tools=[searchDuckDuck],
    verbose=True ##verbose means that the llm produces a details output(not keeping anser concise)
)

In [67]:
response = agent_executor.invoke({'input': "greenland usa news"})



> Entering new AgentExecutor chain...
Thought: The user is asking about "greenland usa news". This could be ambiguous: it might refer to news about Greenland (the territory) in USA news, or perhaps a specific place called Greenland in the USA. I'll search for recent news to clarify.

Action: searchDuckDuck
Action Input: "greenland usa news 2024"13 hours ago - In December 2024, president-elect Trump, before the onset of his second administration, reiterated his interest in the US assuming control of Greenland stating that "for purposes of national security and freedom throughout the world, the United States of America feels that the ownership and ... 2 hours ago - On 9 January 2025, US representative Mike Haridopolos issued a statement supporting US acquisition of Greenland on grounds of space security, saying "it's a critical part of ensuring our nation's security now and in the future ... America cannot afford to cede an inch in space or the Arctic". Earlier, in July 2024, US Army L

In [68]:
print(response)

{'input': 'greenland usa news', 'output': 'The recent "Greenland USA news" primarily revolves around renewed U.S. political interest in acquiring Greenland or securing strategic control over parts of the territory. In late 2024 and early 2025, figures like then-President-elect Donald Trump and U.S. Representative Mike Haridopolos publicly revived the idea, citing national security, Arctic dominance, and access to Greenland\'s rare earth mineral reserves as key reasons. Denmark has opposed the idea, with reports indicating it plans to increase defense spending in Greenland in response. This geopolitical story about U.S. acquisition talks and Arctic strategy is the dominant news topic connecting Greenland and the USA currently.'}


In [69]:
print(response['output'])

The recent "Greenland USA news" primarily revolves around renewed U.S. political interest in acquiring Greenland or securing strategic control over parts of the territory. In late 2024 and early 2025, figures like then-President-elect Donald Trump and U.S. Representative Mike Haridopolos publicly revived the idea, citing national security, Arctic dominance, and access to Greenland's rare earth mineral reserves as key reasons. Denmark has opposed the idea, with reports indicating it plans to increase defense spending in Greenland in response. This geopolitical story about U.S. acquisition talks and Arctic strategy is the dominant news topic connecting Greenland and the USA currently.
